## This File Contains Some Info Reguarding The MAMS For ABSA Repo And How Their Code Works

## This Section Explains Their Preprocess Workflow

Their Whole Config is saved in a yaml format and loaded before preprocessing

In [1]:
import yaml

config = yaml.safe_load(open('config.yml'))

print(type(config))

for key,value in config.items():
    print(key,value, sep=" : " , end='\n')


<class 'dict'>
mode : term
base_path : ./data/MAMS-ATSA
glove_path : ./data/glove.840B.300d.txt
sentiment_path : ./data/sentiment_dict.json
lowercase : True
max_vocab_size : None
min_vocab_freq : 0
aspect_term_model : {'type': 'recurrent_capsnet', 'recurrent_capsnet': {'embed_size': 300, 'dropout': 0.5, 'num_layers': 2, 'capsule_size': 300, 'bidirectional': True, 'optimizer': 'adam', 'batch_size': 64, 'learning_rate': 0.0003, 'weight_decay': 0, 'num_epoches': 20, 'gpu': 0}, 'bert_capsnet': {'bert_size': 768, 'capsule_size': 300, 'dropout': 0.1, 'optimizer': 'adam', 'batch_size': 32, 'learning_rate': 2e-05, 'weight_decay': 0, 'num_epoches': 5, 'gpu': 0}}
aspect_category_model : {'type': 'recurrent_capsnet', 'recurrent_capsnet': {'embed_size': 300, 'dropout': 0.5, 'num_layers': 2, 'capsule_size': 300, 'bidirectional': True, 'optimizer': 'adam', 'batch_size': 64, 'learning_rate': 0.0003, 'weight_decay': 0, 'num_epoches': 20, 'gpu': 0}, 'bert_capsnet': {'bert_size': 768, 'capsule_size': 30

Now They Start The Preprocessing. The Entire Code is Abstracted into a subroutine, but i will unpack some of it here.

In [2]:
# Some Imports
import os
import numpy
import pickle
import yaml


In [3]:
# From The Config File Get the mode
mode = config['mode']
print(mode)

term


In [5]:
# Check Mode in Term or Category
assert mode in ('term', 'category')


In [6]:
#Get the Data Locations
base_path = config['base_path']
raw_train_path = os.path.join(base_path, 'raw/train.xml')
raw_val_path = os.path.join(base_path, 'raw/val.xml')
raw_test_path = os.path.join(base_path, 'raw/test.xml')
print(base_path,raw_train_path,raw_val_path,raw_test_path,sep=" , ",end='\n')

./data/MAMS-ATSA , ./data/MAMS-ATSA\raw/train.xml , ./data/MAMS-ATSA\raw/val.xml , ./data/MAMS-ATSA\raw/test.xml


In [7]:
# Check If The Config Has Lowercase Enabled
lowercase = config['lowercase']
print(lowercase)

True


In [8]:
# In Case Of mode == Term

# This Parses The Raw XML data Into Usable Data

from xml.etree.ElementTree import parse
tree = parse(raw_train_path)
sentences = tree.getroot()
data = []
split_char = '__split__'

print(tree)
print(sentences)
print(type(sentences))
print(data)

for sentence in sentences:
    text = sentence.find('text')
    if text is None:
        continue
    text = text.text
    if lowercase:
        text = text.lower()
    aspectTerms = sentence.find('aspectTerms')
    if aspectTerms is None:
        continue
    print(text, ' ' , end='')
    for aspectTerm in aspectTerms:
        term = aspectTerm.get('term')
        if lowercase:
            term = term.lower()
        polarity = aspectTerm.get('polarity')
        start = aspectTerm.get('from')
        end = aspectTerm.get('to')
        piece = text + split_char + term + split_char + polarity + split_char + start + split_char + end
        print('Term :', term , ' From ',start, ' To ' , end , ' Polarity ' , polarity)
        data.append(piece)
    print('\n')

print(len(sentences))



<Element 'sentences' at 0x0000027574A40BD0>
<class 'xml.etree.ElementTree.Element'>
[]
the decor is not special at all but their food and amazing prices make up for it.  Term : decor  From  4  To  9  Polarity  negative
Term : food  From  42  To  46  Polarity  positive
Term : prices  From  59  To  65  Polarity  positive


when tables opened up, the manager sat another party before us.  Term : tables  From  5  To  11  Polarity  neutral
Term : manager  From  27  To  34  Polarity  negative


though the menu includes some unorthodox offerings (a peanut butter roll, for instance), the classics are pure and great--we've never had better sushi anywhere, including japan.  Term : menu  From  11  To  15  Polarity  neutral
Term : peanut butter roll  From  54  To  72  Polarity  negative
Term : classics  From  93  To  101  Polarity  positive
Term : sushi  From  145  To  150  Polarity  positive


service is good although a bit in your face, we were asked every five mins if food was ok, but better tha

Now If We Add That Code To A Function That Parses The Raw data according to term

In [9]:
from xml.etree.ElementTree import parse

def parse_sentence_term(path, lowercase=False):
    tree = parse(path)
    sentences = tree.getroot()
    data = []
    split_char = '__split__'
    for sentence in sentences:
        text = sentence.find('text')
        if text is None:
            continue
        text = text.text
        if lowercase:
            text = text.lower()
        aspectTerms = sentence.find('aspectTerms')
        if aspectTerms is None:
            continue
        for aspectTerm in aspectTerms:
            term = aspectTerm.get('term')
            if lowercase:
                term = term.lower()
            polarity = aspectTerm.get('polarity')
            start = aspectTerm.get('from')
            end = aspectTerm.get('to')
            piece = text + split_char + term + split_char + polarity + split_char + start + split_char + end
            data.append(piece)
    return data

Now If The Mode Was not selected Term But Category instead

In [10]:
# Parse The Raw XML Data according to Category
from xml.etree.ElementTree import parse

tree = parse(raw_train_path)
sentences = tree.getroot()
data = []
split_char = '__split__'
for sentence in sentences:
    text = sentence.find('text')
    if text is None:
        continue
    text = text.text
    if lowercase:
        text = text.lower()
    aspectCategories = sentence.find('aspectCategories')
    if aspectCategories is None:
        continue
    for aspectCategory in aspectCategories:
        category = aspectCategory.get('category')
        polarity = aspectCategory.get('polarity')
        piece = text + split_char + category + split_char + polarity
        data.append(piece)
print(data)
# Nothing is shown as the data selected is term based

[]


Now As a Function

In [11]:
def parse_sentence_category(path, lowercase=False):
    tree = parse(path)
    sentences = tree.getroot()
    data = []
    split_char = '__split__'
    for sentence in sentences:
        text = sentence.find('text')
        if text is None:
            continue
        text = text.text
        if lowercase:
            text = text.lower()
        aspectCategories = sentence.find('aspectCategories')
        if aspectCategories is None:
            continue
        for aspectCategory in aspectCategories:
            category = aspectCategory.get('category')
            polarity = aspectCategory.get('polarity')
            piece = text + split_char + category + split_char + polarity
            data.append(piece)
    return data


Now Depending On the mode We can parse The data accordingly 

In [12]:
if mode == 'term':
    train_data = parse_sentence_term(raw_train_path, lowercase=lowercase)
    val_data = parse_sentence_term(raw_val_path, lowercase=lowercase)
    test_data = parse_sentence_term(raw_test_path, lowercase=lowercase)
else:
    train_data = parse_sentence_category(raw_train_path, lowercase=lowercase)
    val_data = parse_sentence_category(raw_val_path, lowercase=lowercase)
    test_data = parse_sentence_category(raw_test_path, lowercase=lowercase)

print(len(train_data))
print(len(val_data))
print(len(test_data))

11186
1332
1336


In [13]:
remove_list = ['conflict']
# now We have a Type To Remove
print(remove_list)

['conflict']


In [14]:
# To Remove The Aspects That results in a conflict
remove_set = set(remove_list)
filtered_data = []
print(test_data[0].split('__split__'))
for text in train_data:
    if not text.split('__split__')[2] in remove_set:
        filtered_data.append(text)
print(len(filtered_data))

for i,data in enumerate(filtered_data):
    print(f'{i}th Data is {data}')

["the food was served promptly but the meal wasn't rushed - we had plenty of time to enjoy the appetizers and our entrees as well as sit and chat while finishing up our drinks even after we paid.", 'food', 'neutral', '4', '8']
11186
0th Data is the decor is not special at all but their food and amazing prices make up for it.__split__decor__split__negative__split__4__split__9
1th Data is the decor is not special at all but their food and amazing prices make up for it.__split__food__split__positive__split__42__split__46
2th Data is the decor is not special at all but their food and amazing prices make up for it.__split__prices__split__positive__split__59__split__65
3th Data is when tables opened up, the manager sat another party before us.__split__tables__split__neutral__split__5__split__11
4th Data is when tables opened up, the manager sat another party before us.__split__manager__split__negative__split__27__split__34
5th Data is though the menu includes some unorthodox offerings (a pea

Now If We Make it a Subroutine

In [15]:
def category_filter(data, remove_list):
    remove_set = set(remove_list)
    filtered_data = []
    for text in data:
        if not text.split('__split__')[2] in remove_set:
            filtered_data.append(text)
    return filtered_data

And Filter out the unecessary data from our data

In [16]:
train_data = category_filter(train_data, remove_list)
val_data = category_filter(val_data, remove_list)
test_data = category_filter(test_data, remove_list)
print(len(train_data))
print(len(val_data))
print(len(test_data))


11186
1332
1336


Now The Repo Uses A Custom Class To Make Words 2 Vectors

In [17]:
import operator

# Constrains On The Data
PAD = '<pad>'
UNK = '<unk>'
ASPECT = '<aspect>'

PAD_INDEX = 0
UNK_INDEX = 1
ASPECT_INDEX = 2

INF = 1e9

class Vocab(object):

    def __init__(self):
        self._count_dict = dict()
        self._predefined_list = [PAD, UNK, ASPECT]

    def add(self, word):
        if word in self._count_dict:
            self._count_dict[word] += 1
        else:
            self._count_dict[word] = 1

    def add_list(self, words):
        for word in words:
            self.add(word)

    def get_vocab(self, max_size=None, min_freq=0):
        sorted_words = sorted(self._count_dict.items(), key=operator.itemgetter(1), reverse=True)
        word2index = {}
        for word in self._predefined_list:
            word2index[word] = len(word2index)
        for word, freq in sorted_words:
            if word in word2index:
                continue
            if (max_size is not None and len(word2index) >= max_size) or freq < min_freq:
                word2index[word] = word2index[UNK]
            else:
                word2index[word] = len(word2index)
        index2word = {}
        index2word[word2index[UNK]] = UNK
        for word, index in word2index.items():
            if index == word2index[UNK]:
                continue
            else:
                index2word[index] = word
        return word2index, index2word

Now They Used This Function

In [19]:
import spacy
import re

url = re.compile('(<url>.*</url>)')
# The Repo Used Code Here That was not usuable in the latest library patch and 
# To Do Manual download of the new en_core_web model i needed to do 
# python -m spacy download en_core_web_sm
# that allowed This code to work
spacy_en = spacy.load('en_core_web_sm')

def check(x):
    return len(x) >= 1 and not x.isspace()

def tokenizer(text):
    tokens = [tok.text for tok in spacy_en.tokenizer(url.sub('@URL@', text))]
    return list(filter(check, tokens))

def build_vocab(data, max_size, min_freq):
    if max_size == 'None':
        max_size = None
    vocab = Vocab()
    for piece in data:
        text = piece.split('__split__')[0]
        text = tokenizer(text)
        vocab.add_list(text)
    return vocab.get_vocab(max_size=max_size, min_freq=min_freq)

In [26]:
word2index, index2word = build_vocab(train_data, max_size=config['max_vocab_size'], min_freq=config['min_vocab_freq'])

for x in word2index:
    print(x,end=' , ')

print('\n\n')
for x in index2word:
    print(x, end=' , ')


<pad> , <unk> , <aspect> , the , , , . , and , a , to , of , was , for , i , we , with , is , - , food , in , but , it , our , that , ( , were , ) , on , had , at , you , not , menu , us , n't , service , are , my , they , have , as , waiter , 's , or , bar , dinner , good , be , there , table , when , so , an , like , ! , from , if , after , get , wait , drinks , place , all , out , this , very , by , even , which , one , did , great , wine , -- , waitress , just , up , order , he , only , about , their , no , more , do , $ , time , over , meal , ; , chicken , drink , staff , would , minutes , me , restaurant , while , dessert , your , came , could , other , some , go , than , dishes , appetizers , : , asked , sauce , cheese , price , ordered , lunch , then , can , too , back , waiting , salad , steak , served , dining , well , never , two , made , who , long , entrees , got , because , has , manager , appetizer , went , before , told , room , nice , really , took , tables , small , t

In [25]:
# Make A Folder For the processed Data
if not os.path.exists(os.path.join(base_path, 'processed')):
    os.makedirs(os.path.join(base_path, 'processed'))

Functions To Save Data

In [27]:
import numpy as np
from pytorch_pretrained_bert import BertTokenizer

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def save_term_data(data, word2index, path):
    dirname = os.path.dirname(path)
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    sentence = []
    aspect = []
    label = []
    context = []
    bert_token = []
    bert_segment = []
    td_left = []
    td_right = []
    f = lambda x: word2index[x] if x in word2index else word2index[UNK]
    g = lambda x: list(map(f, tokenizer(x)))
    d = {
        'positive': 0,
        'negative': 1,
        'neutral': 2,
        'conflict': 3
    }
    for piece in data:
        text, term, polarity, start, end = piece.split('__split__')
        start, end = int(start), int(end)
        assert text[start: end] == term
        sentence.append(g(text))
        aspect.append(g(term))
        label.append(d[polarity])
        left_part = g(text[:start])
        right_part = g(text[end:])
        context.append(left_part + [ASPECT_INDEX] + right_part)
        bert_sentence = bert_tokenizer.tokenize(text)
        bert_aspect = bert_tokenizer.tokenize(term)
        bert_token.append(bert_tokenizer.convert_tokens_to_ids(['[CLS]'] + bert_sentence + ['[SEP]'] + bert_aspect + ['[SEP]']))
        bert_segment.append([0] * (len(bert_sentence) + 2) + [1] * (len(bert_aspect) + 1))
        td_left.append(g(text[:end]))
        td_right.append(g(text[start:])[::-1])
        assert len(bert_token[-1]) == len(bert_segment[-1])
    max_length = lambda x: max([len(y) for y in x])
    sentence_max_len = max_length(sentence)
    aspect_max_len = max_length(aspect)
    context_max_len = max_length(context)
    bert_max_len = max_length(bert_token)
    td_left_max_len = max_length(td_left)
    td_right_max_len = max_length(td_right)
    num = len(data)
    for i in range(num):
        sentence[i].extend([0] * (sentence_max_len - len(sentence[i])))
        aspect[i].extend([0] * (aspect_max_len - len(aspect[i])))
        context[i].extend([0] * (context_max_len - len(context[i])))
        bert_token[i].extend([0] * (bert_max_len - len(bert_token[i])))
        bert_segment[i].extend([0] * (bert_max_len - len(bert_segment[i])))
        td_left[i].extend([0] * (td_left_max_len - len(td_left[i])))
        td_right[i].extend([0] * (td_right_max_len - len(td_right[i])))
    sentence = np.asarray(sentence, dtype=np.int32)
    aspect = np.asarray(aspect, dtype=np.int32)
    label = np.asarray(label, dtype=np.int32)
    context = np.asarray(context, dtype=np.int32)
    bert_token = np.asarray(bert_token, dtype=np.int32)
    bert_segment = np.asarray(bert_segment, dtype=np.int32)
    td_left = np.asarray(td_left, dtype=np.int32)
    td_right = np.asarray(td_right, dtype=np.int32)
    np.savez(path, sentence=sentence, aspect=aspect, label=label, context=context, bert_token=bert_token, bert_segment=bert_segment,
             td_left=td_left, td_right=td_right)

def save_category_data(data, word2index, path):
    dirname = os.path.dirname(path)
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    sentence = []
    aspect = []
    label = []
    bert_token = []
    bert_segment = []
    f = lambda x: word2index[x] if x in word2index else word2index[UNK]
    g = lambda x: list(map(f, tokenizer(x)))
    d = {
        'positive': 0,
        'negative': 1,
        'neutral': 2,
        'conflict': 3
    }
    cd = {
        'food': 0,
        'service': 1,
        'staff': 2,
        'price': 3,
        'ambience': 4,
        'menu': 5,
        'place': 6,
        'miscellaneous': 7
    }
    for piece in data:
        text, category, polarity = piece.split('__split__')
        sentence.append(g(text))
        aspect.append(cd[category])
        label.append(d[polarity])
        bert_sentence = bert_tokenizer.tokenize(text)
        bert_aspect = bert_tokenizer.tokenize(category)
        bert_token.append(bert_tokenizer.convert_tokens_to_ids(['[CLS]'] + bert_sentence + ['[SEP]'] + bert_aspect + ['[SEP]']))
        bert_segment.append([0] * (len(bert_sentence) + 2) + [1] * (len(bert_aspect) + 1))
        assert len(bert_token[-1]) == len(bert_segment[-1])
    max_length = lambda x: max([len(y) for y in x])
    sentence_max_len = max_length(sentence)
    bert_max_len = max_length(bert_token)
    num = len(data)
    for i in range(num):
        sentence[i].extend([0] * (sentence_max_len - len(sentence[i])))
        bert_token[i].extend([0] * (bert_max_len - len(bert_token[i])))
        bert_segment[i].extend([0] * (bert_max_len - len(bert_segment[i])))
    sentence = np.asarray(sentence, dtype=np.int32)
    aspect = np.asarray(aspect, dtype=np.int32)
    label = np.asarray(label, dtype=np.int32)
    bert_token = np.asarray(bert_token, dtype=np.int32)
    bert_segment = np.asarray(bert_segment, dtype=np.int32)
    np.savez(path, sentence=sentence, aspect=aspect, label=label, bert_token=bert_token, bert_segment=bert_segment)

In [28]:
if not os.path.exists(os.path.join(base_path, 'processed')):
    os.makedirs(os.path.join(base_path, 'processed'))
    
if mode == 'term':
    save_term_data(train_data, word2index, os.path.join(base_path, 'processed/train.npz'))
    save_term_data(val_data, word2index, os.path.join(base_path, 'processed/val.npz'))
    save_term_data(test_data, word2index, os.path.join(base_path, 'processed/test.npz'))
else:
    save_category_data(train_data, word2index, os.path.join(base_path, 'processed/train.npz'))
    save_category_data(val_data, word2index, os.path.join(base_path, 'processed/val.npz'))
    save_category_data(test_data, word2index, os.path.join(base_path, 'processed/test.npz'))


In [53]:
import json

def load_glove(path, vocab_size, word2index):
    if not os.path.isfile(path):
        raise IOError('Not a file', path)
    glove = np.random.uniform(-0.01, 0.01, [vocab_size, 300])
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            content = line.split(' ')
            if content[0] in word2index:
                glove[word2index[content[0]]] = np.array(list(map(float, content[1:])))
    glove[PAD_INDEX, :] = 0
    return glove

def load_sentiment_matrix(glove_path, sentiment_path):
    sentiment_matrix = np.zeros((3, 300), dtype=np.float32)
    sd = json.load(open(sentiment_path, 'r', encoding='utf-8'))
    sd['positive'] = set(sd['positive'])
    sd['negative'] = set(sd['negative'])
    sd['neutral'] = set(sd['neutral'])
    with open(glove_path, 'r', encoding='utf-8') as f:
        for line in f:
            content = line.split(' ')
            word = content[0]
            vec = np.array(list(map(float, content[1:])))
            if word in sd['positive']:
                sentiment_matrix[0] += vec
            elif word in sd['negative']:
                sentiment_matrix[1] += vec
            elif word in sd['neutral']:
                sentiment_matrix[2] += vec
    sentiment_matrix -= sentiment_matrix.mean()
    sentiment_matrix = sentiment_matrix / sentiment_matrix.std() * np.sqrt(2.0 / (300.0 + 3.0))
    return sentiment_matrix

def analyze_term(data):
    num = len(data)
    sentence_lens = []
    aspect_lens = []
    log = {'total': num}
    for piece in data:
        text, term, polarity, _, _ = piece.split('__split__')
        sentence_lens.append(len(tokenizer(text)))
        aspect_lens.append(len(tokenizer(term)))
        if not polarity in log:
            log[polarity] = 0
        log[polarity] += 1
    log['sentence_max_len'] = max(sentence_lens)
    log['sentence_avg_len'] = sum(sentence_lens) / len(sentence_lens)
    log['aspect_max_len'] = max(aspect_lens)
    log['aspect_avg_len'] = sum(aspect_lens) / len(aspect_lens)
    return log

def analyze_category(data):
    num = len(data)
    sentence_lens = []
    log = {'total': num}
    for piece in data:
        text, category, polarity = piece.split('__split__')
        sentence_lens.append(len(tokenizer(text)))
        if not polarity in log:
            log[polarity] = 0
        log[polarity] += 1
    log['sentence_max_len'] = max(sentence_lens)
    log['sentence_avg_len'] = sum(sentence_lens) / len(sentence_lens)
    return log


In [54]:
glove = load_glove(config['glove_path'], len(index2word), word2index)
sentiment_matrix = load_sentiment_matrix(config['glove_path'], config['sentiment_path'])
np.save(os.path.join(base_path, 'processed/glove.npy'), glove)
np.save(os.path.join(base_path, 'processed/sentiment_matrix.npy'), sentiment_matrix)
with open(os.path.join(base_path, 'processed/word2index.pickle'), 'wb') as handle:
    pickle.dump(word2index, handle)
with open(os.path.join(base_path, 'processed/index2word.pickle'), 'wb') as handle:
    pickle.dump(index2word, handle)
analyze = analyze_term if mode == 'term' else analyze_category
log = {
    'vocab_size': len(index2word),
    'oov_size': len(word2index) - len(index2word),
    'train_data': analyze(train_data),
    'val_data': analyze(val_data),
    'test_data': analyze(test_data),
    'num_categories': 3
}
if not os.path.exists(os.path.join(base_path, 'log')):
    os.makedirs(os.path.join(base_path, 'log'))
with open(os.path.join(base_path, 'log/log.yml'), 'w') as handle:
    yaml.safe_dump(log, handle, encoding='utf-8', allow_unicode=True, default_flow_style=False)